In [1]:
import numpy as np

In [2]:
def power_iteration(
        A: np.ndarray,
        n_max_steps: int = 100000,
        convergence_threshold: float = 1e-10,
        x_init: np.ndarray = None,
        normalize : bool = False
):
    n, m = A.shape
    
    if n != m:
        raise ValueError("the matrix A must be square")
    
    if x_init is not None:
        x = x_init.reshape(-1, 1)
    else:
        x = np.random.normal(size=(n,1))

    for step in range(n_max_steps):
        x_transformed = A @ x
        x_new = x_transformed / np.linalg.norm(x_transformed, ord=np.inf)

        diff = np.linalg.norm(x - x_new)
        x = x_new

        if diff < convergence_threshold:
            break
    
    if normalize:
        return x / np.linalg.norm(x)
    
    return x

In [3]:
A = np.array([[2, 1], [1, 2]])
u_1 = power_iteration(A, normalize=True)

In [4]:
u_1

array([[0.70710678],
       [0.70710678]])

In [5]:
A @ u_1 / u_1

array([[3.],
       [3.]])

In [7]:
def get_orthogonal_complement_projection(u: np.ndarray):
    
    u = u.reshape(-1, 1)
    n, _ = u.shape
    return np.eye(n) -u @ u.T / np.linalg.norm(u, ord=2)**2

In [8]:
def find_eigenvectors(A: np.ndarray, x_init: np.ndarray):
    
    n, _ = A.shape
    eigenvectors = []

    for _ in range(n):
        ev = power_iteration(A, x_init=x_init)
        proj = get_orthogonal_complement_projection(ev)
        x_init = proj @ x_init
        x_init = x_init / np.linalg.norm(x_init, ord = np.inf)
        eigenvectors.append(ev)

    return eigenvectors

In [9]:
A = np.array([[2.0, 1.0], [1.0, 2.0]])
x_init = np.random.rand(2, 1)

In [10]:
find_eigenvectors(A, x_init)

[array([[1.],
        [1.]]),
 array([[-1.],
        [ 1.]])]

In [11]:
def diagonalize_symmetric_matrix(A: np.ndarray, x_init: np.ndarray):
    
    eigenvectors = find_eigenvectors(A, x_init)
    U = np.hstack(eigenvectors) / np.linalg.norm(np.hstack(eigenvectors), axis=0, ord=2)
    return U, U @ A @ U.T

In [12]:
diagonalize_symmetric_matrix(A, x_init)

(array([[ 0.70710678, -0.70710678],
        [ 0.70710678,  0.70710678]]),
 array([[ 1.00000000e+00, -5.89253279e-11],
        [-5.89253956e-11,  3.00000000e+00]]))

In [13]:
def projection_coeff(x: np.ndarray, to: np.ndarray):

    return np.dot(x, to) / np.dot(to, to)

In [14]:
from typing import List

def projection(x: np.ndarray, to: List[np.ndarray], return_coeffs: bool = True):

    p_x = np.zeros_like(x)
    coeffs = []

    for e in to:
        coeff = projection_coeff(x, e)
        coeffs.append(coeff)
        p_x += coeff * e

    if return_coeffs:
        return p_x, coeffs
    else:
        return p_x

In [16]:
def QR(A: np.ndarray):

    n, m = A.shape
    
    A_columns = [A[:, i] for i in range(A.shape[1])]
    Q_columns, R_columns = [], []

    Q_columns.append(A_columns[0])
    R_columns.append([1] + (m-1)*[0])

    for i, a in enumerate(A_columns[1:]):
        p, coeffs = projection(a, Q_columns, return_coeffs=True)
        next_q = a - p
        next_r = coeffs + [1] + max(0, m - i - 2)*[0]

        Q_columns.append(next_q)
        R_columns.append(next_r)

    Q, R = np.array(Q_columns).T, np.array(R_columns).T

    Q_norms = np.linalg.norm(Q, axis=0)
    Q = Q/Q_norms
    R = np.diag(Q_norms) @ R
    return Q, R

In [17]:
A = np.random.rand(3, 3)
Q, R = QR(A)

In [19]:
np.allclose(A, Q @ R)

True

In [18]:
np.allclose(Q.T @ Q, np.eye(3))

True

In [20]:
np.allclose(R, np.triu(R))

True

In [21]:
def QR_algorithm(A: np.ndarray, n_iter: int = 1000):

    for _ in range(n_iter):
        Q, R = QR(A)
        A = R @ Q

    return A

In [22]:

A = np.array([[2.0, 1.0], [1.0, 2.0]])
QR_algorithm(A)

array([[3.00000000e+00, 2.39107046e-16],
       [0.00000000e+00, 1.00000000e+00]])

In [23]:
A = np.array([[0.0, 1.0], [1.0, 0.0]])
QR_algorithm(A)

array([[0., 1.],
       [1., 0.]])